## Una vez más, modelaremos los precios de viviendas

### Vamos a generar un ejemplo de Pipe. Primero generemos un breve EDA

In [4]:
#!pip install plotly

In [6]:
import pandas as pd
import numpy as np
#import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px

## Generemos un breve EDA

In [9]:
df = pd.read_csv('https://raw.githubusercontent.com/laulb/Reto_Sesion_7/main/data.csv')
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [10]:
# % de Habitaciones de las casas
rooms = df['Rooms'].value_counts().sort_index()

fig = px.pie(rooms, values=rooms.values, names=rooms.index,
             title='A number of rooms in the houses %',
             color_discrete_sequence=px.colors.sequential.RdBu)

fig.show()

In [11]:
# numero de casas por suburbios
suburb = df.Suburb.value_counts()
others = suburb[30:].sum()
suburb = suburb[:30]

fig = px.bar(suburb, y=suburb.values, x=suburb.index, text=suburb.values, color=suburb.index,
             labels={'y': 'A number of houses', 'index': 'Name of the suburb'},
             title='Top 30 suburbs of Melbourne',
             color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

## Ahora empecemos con el metodo pipeline pero primer particionemos los datos

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [17]:
# Separar predictores de variable a predecir
y = df.Price
X = df.drop(['Price'], axis=1)

# Hacer partición de training y tetsing
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Nos interesan las variables categoricas con pocos unique values (no queremos infinidad de categorías)
# Para eso solo queremos variables con maximo 10 unique values
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# seleccionar solo las numericas
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Quedarse solo con las variables seleccionadas
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [18]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


## Paso tercero: concluir y evaluar el pipe
### Finalmente, usamos  Pipeline para agrupar los pasos de preprocesamiento y modelado. veamos:

Con el pipeline, preprocesamos los datos de entrenamiento y ajustamos el modelo en una sola línea de código. (Por el contrario, sin una canalización, tenemos que realizar la definición, la codificación one-hot y el entrenamiento del modelo en pasos separados, lo cual se vuelve complicado si tenemos que lidiar con variables tanto numéricas como categóricas

Con el pipeline, proporcionamos las características no procesadas en X_valid al comando predict (), y el pipeline preprocesa automáticamente las características antes de generar predicciones. (Sin embargo, sin un pipeline, debemos recordar preprocesar los datos de validación antes de hacer predicciones).

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocesar datos numericos
numerical_transformer = SimpleImputer(strategy='constant')

# preprocesar datos categoricos
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])



# Juntar variables categoricas y numericas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [20]:
# Definir el Random Forest

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [ ]:
from sklearn.metrics import mean_absolute_error

# Conjuntar los datos preprocesados y hacer el fit
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Enbonar el training
my_pipeline.fit(X_train, y_train)

# Adjuntar el training data y terminar el fit
preds = my_pipeline.predict(X_valid)

# Siguiente paso: evaluación del modelo
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 160679.18917034855
